In [2]:
!!pip install MOABB

['Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/',
 'Collecting MOABB',
 '  Downloading moabb-0.4.6-py3-none-any.whl (140 kB)',
 '\x1b[?25l',
 '\x1b[K     |██▎                             | 10 kB 19.9 MB/s eta 0:00:01',
 '\x1b[K     |████▋                           | 20 kB 7.2 MB/s eta 0:00:01',
 '\x1b[K     |███████                         | 30 kB 10.0 MB/s eta 0:00:01',
 '\x1b[K     |█████████▎                      | 40 kB 4.6 MB/s eta 0:00:01',
 '\x1b[K     |███████████▋                    | 51 kB 4.4 MB/s eta 0:00:01',
 '\x1b[K     |██████████████                  | 61 kB 5.2 MB/s eta 0:00:01',
 '\x1b[K     |████████████████▎               | 71 kB 5.6 MB/s eta 0:00:01',
 '\x1b[K     |██████████████████▋             | 81 kB 6.3 MB/s eta 0:00:01',
 '\x1b[K     |█████████████████████           | 92 kB 6.6 MB/s eta 0:00:01',
 '\x1b[K     |███████████████████████▎        | 102 kB 5.3 MB/s eta 0:00:01',
 '\x1b[K     |███████████████████

In [85]:
import numpy as np
import scipy.signal
import scipy.io as sio
import matplotlib
from matplotlib import pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

In [3]:
import moabb
from moabb.datasets import EPFLP300, utils
from moabb.evaluations import CrossSessionEvaluation
from moabb.paradigms import LeftRightImagery
from moabb.pipelines.features import LogVariance
moabb.set_log_level("info")


In [4]:
def getData(subjectNumbers,dataset=EPFLP300()):
  return dataset.get_data(subjectNumbers)

In [94]:
def getDataFrame(subjectNumber,session_number,run_number,resample=None):
  session = f'session_{session_number}'
  run = f'run_{run_number}'
  data = getData([subjectNumber])
  if resample!=None: return data[subjectNumber][session][run].resample(resample).to_data_frame()
  return  data[subjectNumber][session][run].to_data_frame()

In [95]:
data=getDataFrame(1,1,1)

In [89]:
data.head()


,time,Fp1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,...,FC2,F4,F8,AF4,Fp2,Fz,Cz,MA1,MA2,STI
0,0.000000,3566.625000,27146.750000,-9512.343750,19556.281250,1640.281250,-1500.781250,-8068.000000,3436.687500,-12713.562500,...,7099.187500,-2389.000000,-16210.187500,4854.968750,22967.625000,6985.312500,8299.875000,0.006680,-0.006680,0.0
1,0.000488,3564.203125,27144.609375,-9515.796875,19553.046875,1638.640625,-1502.734375,-8072.609375,3434.109375,-12718.734375,...,7095.828125,-2392.078125,-16213.421875,4852.578125,22965.609375,6981.921875,8297.671875,0.006677,-0.006677,0.0
2,0.000977,3561.390625,27141.890625,-9519.296875,19548.890625,1635.734375,-1504.921875,-8076.140625,3433.578125,-12722.203125,...,7092.609375,-2395.671875,-16216.046875,4850.046875,22964.734375,6980.421875,8295.234375,0.006671,-0.006671,0.0
3,0.001465,3559.515625,27139.015625,-9521.640625,19546.609375,1632.765625,-1505.546875,-8077.828125,3430.984375,-12722.890625,...,7089.953125,-2398.734375,-16219.546875,4845.796875,22962.515625,6979.203125,8293.765625,0.006667,-0.006667,0.0
4,0.001953,3556.421875,27137.296875,-9523.734375,19546.296875,1631.390625,-1507.171875,-8081.390625,3430.109375,-12724.953125,...,7087.453125,-2400.859375,-16222.421875,4842.890625,22960.390625,6977.453125,8291.078125,0.006665,-0.006665,0.0


In [38]:
data


,time,Fp1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,...,FC2,F4,F8,AF4,Fp2,Fz,Cz,MA1,MA2,STI
0,0.000000,3566.625000,27146.750000,-9512.343750,19556.281250,1640.281250,-1500.781250,-8068.000000,3436.687500,-12713.562500,...,7099.187500,-2389.000000,-16210.187500,4854.968750,22967.625000,6985.312500,8299.875000,0.006680,-0.006680,0.0
1,0.000488,3564.203125,27144.609375,-9515.796875,19553.046875,1638.640625,-1502.734375,-8072.609375,3434.109375,-12718.734375,...,7095.828125,-2392.078125,-16213.421875,4852.578125,22965.609375,6981.921875,8297.671875,0.006677,-0.006677,0.0
2,0.000977,3561.390625,27141.890625,-9519.296875,19548.890625,1635.734375,-1504.921875,-8076.140625,3433.578125,-12722.203125,...,7092.609375,-2395.671875,-16216.046875,4850.046875,22964.734375,6980.421875,8295.234375,0.006671,-0.006671,0.0
3,0.001465,3559.515625,27139.015625,-9521.640625,19546.609375,1632.765625,-1505.546875,-8077.828125,3430.984375,-12722.890625,...,7089.953125,-2398.734375,-16219.546875,4845.796875,22962.515625,6979.203125,8293.765625,0.006667,-0.006667,0.0
4,0.001953,3556.421875,27137.296875,-9523.734375,19546.296875,1631.390625,-1507.171875,-8081.390625,3430.109375,-12724.953125,...,7087.453125,-2400.859375,-16222.421875,4842.890625,22960.390625,6977.453125,8291.078125,0.006665,-0.006665,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114682,55.997070,3530.156250,27568.593750,-10146.500000,19951.156250,1380.687500,-455.531250,-8331.031250,3226.156250,-12875.937500,...,6766.250000,-2992.875000,-17246.656250,4532.218750,23213.656250,6191.156250,8356.093750,0.006414,-0.006414,0.0
114683,55.997559,3526.687500,27566.281250,-10151.531250,19947.906250,1376.312500,-459.531250,-8334.875000,3221.906250,-12879.031250,...,6761.750000,-2997.468750,-17251.468750,4530.093750,23209.468750,6186.343750,8352.718750,0.006414,-0.006414,0.0
114684,55.998047,3522.703125,27564.140625,-10153.359375,19945.796875,1372.734375,-462.015625,-8337.953125,3217.453125,-12883.265625,...,6758.140625,-2999.609375,-17260.046875,4524.984375,23207.359375,6182.796875,8349.890625,0.006413,-0.006413,0.0
114685,55.998535,3524.187500,27564.968750,-10151.437500,19946.687500,1373.281250,-461.093750,-8337.000000,3218.843750,-12884.906250,...,6759.375000,-3000.281250,-17261.968750,4524.187500,23207.906250,6183.468750,8349.250000,0.006412,-0.006412,0.0


In [181]:
def bandPassFilter(data, sampligFrequency, lowFrequency, highFrequency, order):
  sos=scipy.signal.butter(order, Wn=[lowFrequency,highFrequency],btype='bandpass', fs=sampligFrequency, output='sos')
  filteredData = scipy.signal.sosfilt(sos, data)
 #plotting
  # t = np.linspace(0, 1, 114687 , False)  # 1 second
  # fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
  # ax1.plot(t, filteredData)
  # ax1.set_title('10 Hz and 20 Hz sinusoids')
  # ax1.axis([0, 1,1, 20])

  # ax2.plot(t, filteredData)
  # ax2.set_title('After 15 Hz high-pass filter')
  # ax2.axis([0, 1,1, 20])
  # ax2.set_xlabel('Time [seconds]')
  # plt.tight_layout()
  # plt.show()
  return filteredData

In [182]:
filtered_data=bandPassFilter(data,2048,1,12,4)
filtered_data

array([[0.00000000e+00, 2.76772030e-04, 4.29624800e-03, ...,
        6.48022032e+01, 6.75929879e+01, 7.03684794e+01],
       [3.78908892e-11, 2.76584390e-04, 4.29459622e-03, ...,
        6.46945154e+01, 6.74735311e+01, 7.02365125e+01],
       [7.57817783e-11, 2.76366439e-04, 4.29265976e-03, ...,
        6.45966019e+01, 6.73650711e+01, 7.01168319e+01],
       ...,
       [4.34547873e-06, 3.07742352e-04, 4.43727720e-03, ...,
        6.31806869e+01, 6.55891015e+01, 6.79310516e+01],
       [4.34551662e-06, 3.07857840e-04, 4.43825395e-03, ...,
        6.32122596e+01, 6.56239573e+01, 6.79693660e+01],
       [4.34555451e-06, 3.07569563e-04, 4.43570776e-03, ...,
        6.30968598e+01, 6.54964779e+01, 6.78290253e+01]])

In [175]:
# time=np.linspace(0,0.00001,114687, 36)
# plt.figure(figsize=(20, 20))
# plt.plot(time,filteredData)
# plt.show()